In [1]:
import numpy as np
import pandas as pd

In [2]:
ddos = pd.read_csv('DDos_all.csv')

In [3]:
ddos.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'FlowDuration',
       'TotalFwdPackets', 'TotalBackwardPackets', 'TotalLengthofFwdPackets',
       'TotalLengthofBwdPackets', 'FwdPacketLengthMax', 'FwdPacketLengthMin',
       'FwdPacketLengthMean', 'FwdPacketLengthStd', 'BwdPacketLengthMax',
       'BwdPacketLengthMin', 'BwdPacketLengthMean', 'BwdPacketLengthStd',
       'FlowBytes/s', 'FlowPackets/s', 'FlowIATMean', 'FlowIATStd',
       'FlowIATMax', 'FlowIATMin', 'FwdIATTotal', 'FwdIATMean', 'FwdIATStd',
       'FwdIATMax', 'FwdIATMin', 'BwdIATTotal', 'BwdIATMean', 'BwdIATStd',
       'BwdIATMax', 'BwdIATMin', 'FwdPSHFlags', 'BwdPSHFlags', 'FwdURGFlags',
       'BwdURGFlags', 'FwdHeaderLength', 'BwdHeaderLength', 'FwdPackets/s',
       'BwdPackets/s', 'MinPacketLength', 'MaxPacketLength',
       'PacketLengthMean', 'PacketLengthStd', 'PacketLengthVariance',
       'FINFlagCount', 'SYNFlagCount', 'RSTFlagCount', 'PSHFlagCount',
       'ACKFlagCount', 'URGFlagCount', 'CWEFlagCou

In [4]:
ddos = ddos.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1'])

In [6]:
ddos['Label'].value_counts()

MSSQL      5787453
Syn        4284751
UDP        3754680
NetBIOS    3657497
LDAP       1915122
Portmap     186960
BENIGN       52897
Name: Label, dtype: int64

In [9]:
ddos['Label'].replace('BENIGN',0,inplace = True)
ddos['Label'].replace('UDP',4,inplace = True)
ddos['Label'].replace('LDAP',5,inplace = True)
ddos['Label'].replace('Syn',3,inplace = True)
ddos['Label'].replace('MSSQL',2,inplace = True)
ddos['Label'].replace('Portmap',6,inplace = True)
ddos['Label'].replace('NetBIOS',1,inplace = True)

In [11]:
ddos['Label'].value_counts()

2    5787453
3    4284751
4    3754680
1    3657497
5    1915122
6     186960
0      52897
Name: Label, dtype: int64

In [12]:
cat_columns = ddos.select_dtypes(['object']).columns
ddos[cat_columns] = ddos[cat_columns].astype('category')
ddos[cat_columns] = ddos[cat_columns].apply(lambda x: x.cat.codes)
ddos.dtypes

FlowDuration               float64
TotalFwdPackets            float64
TotalBackwardPackets       float64
TotalLengthofFwdPackets    float64
TotalLengthofBwdPackets    float64
                            ...   
IdleMean                   float64
IdleStd                    float64
IdleMax                    float64
IdleMin                    float64
Label                        int64
Length: 78, dtype: object

In [13]:
X = ddos.iloc[:, 0:-1].values
y = ddos.iloc[:, -1].values

In [14]:
ddos = None; del ddos

In [15]:
from sklearn.preprocessing import StandardScaler
standardScaler = StandardScaler()
X= standardScaler.fit_transform(X)
X

array([[-0.12880758,  0.00808122,  0.21245269, ..., -0.12429176,
        -0.14826039, -0.13491141],
       [-0.15248456, -0.00550824,  0.21245269, ..., -0.12429176,
        -0.14826039, -0.13491141],
       [-0.13130602,  0.00808122,  0.21245269, ..., -0.12429176,
        -0.14826039, -0.13491141],
       ...,
       [-0.1524877 , -0.01230297,  0.21245269, ..., -0.12429176,
        -0.14826039, -0.13491141],
       [-0.1524872 , -0.01230297,  0.21245269, ..., -0.12429176,
        -0.14826039, -0.13491141],
       [-0.14469833,  0.00808122,  0.21245269, ..., -0.12429176,
        -0.14826039, -0.13491141]])

In [16]:
from keras.utils import np_utils
X=np.expand_dims(X,axis=2)
X.shape

(19639360, 77, 1)

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)

# No Freeze

In [18]:
from tensorflow import keras
model = keras.models.load_model('Dos_8output.h5')

In [19]:
from keras.callbacks import EarlyStopping
import time

start = time.time()
print("Timer Begin")

model.fit(X_train,Y_train,batch_size=30,epochs=5,
          validation_split=0, callbacks=[EarlyStopping(monitor='loss',patience=2)])

end = time.time()
print(end - start)

Timer Begin
Epoch 1/5
438613/438613 [==============================] - 2875s 7ms/step - loss: 0.1181 - accuracy: 0.9712
Epoch 2/5
438613/438613 [==============================] - 3348s 8ms/step - loss: 0.1142 - accuracy: 0.9730
Epoch 3/5
438613/438613 [==============================] - 3852s 9ms/step - loss: 0.1225 - accuracy: 0.9727
Epoch 4/5
438613/438613 [==============================] - 4164s 9ms/step - loss: 0.1292 - accuracy: 0.9730
14284.227603435516


In [20]:
y_pred = model.predict(X_test)
y_pred = y_pred.argmax(axis = 1)

In [21]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [22]:
rf_conf_mat = confusion_matrix(Y_test, y_pred)
print(rf_conf_mat)

[[  17210      20       0      87       0       3       0       0]
 [    148 1204178     271      20    2792       4       0       0]
 [    152      23 1837540      37   29373   42269       0       0]
 [    870      13    1464 1411145     235       7       0       3]
 [     17     105   24883       8 1213729      73       0       0]
 [    105      23    7177      42       0  625442       0       0]
 [    118   60921     259      63     157       2       0       1]
 [      0       0       0       0       0       0       0       0]]


In [23]:
CR = classification_report(Y_test,y_pred)
print(CR)

C:\Users\LabUser\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\LabUser\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\LabUser\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\LabUser\anaconda3\lib\site-packages\sklearn\metr

              precision    recall  f1-score   support

           0       0.92      0.99      0.96     17320
           1       0.95      1.00      0.97   1207413
           2       0.98      0.96      0.97   1909394
           3       1.00      1.00      1.00   1413737
           4       0.97      0.98      0.98   1238815
           5       0.94      0.99      0.96    632789
           6       0.00      0.00      0.00     61521
           7       0.00      0.00      0.00         0

    accuracy                           0.97   6480989
   macro avg       0.72      0.74      0.73   6480989
weighted avg       0.96      0.97      0.97   6480989



C:\Users\LabUser\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\LabUser\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Freeze the first layer

In [24]:
model = keras.models.load_model('Dos_8output.h5')

In [25]:
model.layers[0].trainable = False

In [26]:
from keras.callbacks import EarlyStopping
import time

start = time.time()
print("Timer Begin")

model.fit(X_train,Y_train,batch_size=30,epochs=5,
          validation_split=0, callbacks=[EarlyStopping(monitor='loss',patience=2)])

end = time.time()
print(end - start)

Timer Begin
Epoch 1/5
438613/438613 [==============================] - 4820s 11ms/step - loss: 0.1172 - accuracy: 0.9713
Epoch 2/5
438613/438613 [==============================] - 5338s 12ms/step - loss: 0.1160 - accuracy: 0.9731
Epoch 3/5
438613/438613 [==============================] - 6480s 15ms/step - loss: 0.1166 - accuracy: 0.9729
Epoch 4/5
438613/438613 [==============================] - 6498s 15ms/step - loss: 0.1168 - accuracy: 0.9730
23175.89872288704


In [27]:
y_pred = model.predict(X_test)
y_pred = y_pred.argmax(axis = 1)

In [28]:
rf_conf_mat = confusion_matrix(Y_test, y_pred)
print(rf_conf_mat)

[[  17202      23       0      95       0       0       0]
 [    557 1203803     270      22    2761       0       0]
 [    162      21 1836463      40   31282   41426       0]
 [    817      13    1464 1411208     235       0       0]
 [     17     109   24232       8 1214449       0       0]
 [    126       1   10449      43       4  622166       0]
 [    150   60892     259      63     157       0       0]]


In [29]:
CR = classification_report(Y_test,y_pred)
print(CR)

C:\Users\LabUser\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\LabUser\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.90      0.99      0.95     17320
           1       0.95      1.00      0.97   1207413
           2       0.98      0.96      0.97   1909394
           3       1.00      1.00      1.00   1413737
           4       0.97      0.98      0.98   1238815
           5       0.94      0.98      0.96    632789
           6       0.00      0.00      0.00     61521

    accuracy                           0.97   6480989
   macro avg       0.82      0.84      0.83   6480989
weighted avg       0.96      0.97      0.97   6480989



C:\Users\LabUser\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Freeze first two layers

In [30]:
model = keras.models.load_model('Dos_8output.h5')

In [31]:
model.layers[0].trainable = False
model.layers[1].trainable = False

In [32]:
from keras.callbacks import EarlyStopping
import time

start = time.time()
print("Timer Begin")

model.fit(X_train,Y_train,batch_size=30,epochs=5,
          validation_split=0, callbacks=[EarlyStopping(monitor='loss',patience=2)])

end = time.time()
print(end - start)

Timer Begin
Epoch 1/5
438613/438613 [==============================] - 5989s 14ms/step - loss: 0.1188 - accuracy: 0.9713
Epoch 2/5
438613/438613 [==============================] - 6395s 15ms/step - loss: 0.1171 - accuracy: 0.9731
Epoch 3/5
438613/438613 [==============================] - 6623s 15ms/step - loss: 0.1198 - accuracy: 0.9731
Epoch 4/5
438613/438613 [==============================] - 6628s 15ms/step - loss: 0.1244 - accuracy: 0.9729
25678.278839111328


In [33]:
model.save('trans_DDtoD_2.h5')

In [34]:
y_pred = model.predict(X_test)
y_pred = y_pred.argmax(axis=1)

In [35]:
rf_conf_mat = confusion_matrix(Y_test, y_pred)
print("Transfer learning confusion matrix: \n", rf_conf_mat)

Transfer learning confusion matrix: 
 [[  16772     314       9     224       0       1       0]
 [     30 1205082     270      21    2010       0       0]
 [     86      96 1834402      39   31140   43631       0]
 [    729      97    1465 1411211     235       0       0]
 [      7     128   24196       8 1214476       0       0]
 [     23     105    6761      42       4  625854       0]
 [     39   61053     259      68     102       0       0]]


In [36]:
CR = classification_report(Y_test,y_pred)
print(CR)

C:\Users\LabUser\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\LabUser\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.95      0.97      0.96     17320
           1       0.95      1.00      0.97   1207413
           2       0.98      0.96      0.97   1909394
           3       1.00      1.00      1.00   1413737
           4       0.97      0.98      0.98   1238815
           5       0.93      0.99      0.96    632789
           6       0.00      0.00      0.00     61521

    accuracy                           0.97   6480989
   macro avg       0.83      0.84      0.83   6480989
weighted avg       0.96      0.97      0.97   6480989



C:\Users\LabUser\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Freeze first three Layers

In [37]:
model = keras.models.load_model('Dos_8output.h5')

In [38]:
model.layers[0].trainable = False
model.layers[1].trainable = False
model.layers[2].trainable = False

In [39]:
from keras.callbacks import EarlyStopping
import time

start = time.time()
print("Timer Begin")

model.fit(X_train,Y_train,batch_size=30,epochs=5,
          validation_split=0, callbacks=[EarlyStopping(monitor='loss',patience=2)])

end = time.time()
print(end - start)

Timer Begin
Epoch 1/5
438613/438613 [==============================] - 6142s 14ms/step - loss: 0.1183 - accuracy: 0.9712
Epoch 2/5
438613/438613 [==============================] - 6503s 15ms/step - loss: 0.1147 - accuracy: 0.9728
Epoch 3/5
438613/438613 [==============================] - 6613s 15ms/step - loss: 0.1398 - accuracy: 0.9727
Epoch 4/5
438613/438613 [==============================] - 6570s 15ms/step - loss: 0.1378 - accuracy: 0.9728
25872.663601398468


In [40]:
model.save('trans_DDtoD_3.h5')

In [41]:
y_pred = model.predict(X_test)
y_pred = y_pred.argmax(axis=1)

In [42]:
rf_conf_mat = confusion_matrix(Y_test, y_pred)
print("Transfer learning confusion matrix: \n", rf_conf_mat)

Transfer learning confusion matrix: 
 [[  16970     115       0     235       0       0       0]
 [     56 1205352     270      22    1713       0       0]
 [    107      74 1851079      42   16759   41333       0]
 [    652     136    1485 1411250     214       0       0]
 [      9     132   25079       8 1213587       0       0]
 [     19     101    7635      50       1  624983       0]
 [     47   61062     259      66      87       0       0]]


In [43]:
CR = classification_report(Y_test,y_pred)
print(CR)

C:\Users\LabUser\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\LabUser\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.95      0.98      0.96     17320
           1       0.95      1.00      0.97   1207413
           2       0.98      0.97      0.98   1909394
           3       1.00      1.00      1.00   1413737
           4       0.98      0.98      0.98   1238815
           5       0.94      0.99      0.96    632789
           6       0.00      0.00      0.00     61521

    accuracy                           0.98   6480989
   macro avg       0.83      0.84      0.84   6480989
weighted avg       0.97      0.98      0.97   6480989



C:\Users\LabUser\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Freeze the first four Layers

In [44]:
model = keras.models.load_model('Dos_8output.h5')

In [45]:
model.layers[0].trainable = False
model.layers[1].trainable = False
model.layers[2].trainable = False
model.layers[3].trainable = False

In [46]:
from keras.callbacks import EarlyStopping
import time

start = time.time()
print("Timer Begin")

model.fit(X_train,Y_train,batch_size=30,epochs=5,
          validation_split=0, callbacks=[EarlyStopping(monitor='loss',patience=2)])

end = time.time()
print(end - start)

Timer Begin
Epoch 1/5
438613/438613 [==============================] - 6069s 14ms/step - loss: 0.1170 - accuracy: 0.9713
Epoch 2/5
438613/438613 [==============================] - 6434s 15ms/step - loss: 0.1152 - accuracy: 0.9729
Epoch 3/5
438613/438613 [==============================] - 6591s 15ms/step - loss: 0.1166 - accuracy: 0.9728
Epoch 4/5
438613/438613 [==============================] - 6660s 15ms/step - loss: 0.1280 - accuracy: 0.9727
25796.754292964935


In [47]:
model.save('trans_DDtoD_4.h5')

In [48]:
y_pred = model.predict(X_test)
y_pred = y_pred.argmax(axis=1)

In [49]:
rf_conf_mat = confusion_matrix(Y_test, y_pred)
print("Transfer learning confusion matrix: \n", rf_conf_mat)

Transfer learning confusion matrix: 
 [[  17274      24       0      17       1       4       0]
 [    333 1204964     270      95    1751       0       0]
 [    149      24 1853585      38   14754   40844       0]
 [   1105      13    1485 1410920     214       0       0]
 [     15     164   27708      10 1210917       1       0]
 [    135       1   15802      34       3  616814       0]
 [     84   60963     260     125      89       0       0]]


In [50]:
CR = classification_report(Y_test,y_pred)
print(CR)

C:\Users\LabUser\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\LabUser\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.90      1.00      0.95     17320
           1       0.95      1.00      0.97   1207413
           2       0.98      0.97      0.97   1909394
           3       1.00      1.00      1.00   1413737
           4       0.99      0.98      0.98   1238815
           5       0.94      0.97      0.96    632789
           6       0.00      0.00      0.00     61521

    accuracy                           0.97   6480989
   macro avg       0.82      0.85      0.83   6480989
weighted avg       0.97      0.97      0.97   6480989



C:\Users\LabUser\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Freeze the first five layers

In [51]:
model = keras.models.load_model('Dos_8output.h5')

In [52]:
model.layers[0].trainable = False
model.layers[1].trainable = False
model.layers[2].trainable = False
model.layers[3].trainable = False
model.layers[4].trainable = False

In [53]:
from keras.callbacks import EarlyStopping
import time

start = time.time()
print("Timer Begin")

model.fit(X_train,Y_train,batch_size=30,epochs=5,
          validation_split=0, callbacks=[EarlyStopping(monitor='loss',patience=2)])

end = time.time()
print(end - start)

Timer Begin
Epoch 1/5
438613/438613 [==============================] - 6426s 15ms/step - loss: 0.1192 - accuracy: 0.9714
Epoch 2/5
438613/438613 [==============================] - 6756s 15ms/step - loss: 0.1175 - accuracy: 0.9730
Epoch 3/5
438613/438613 [==============================] - 6945s 16ms/step - loss: 0.1167 - accuracy: 0.9729
Epoch 4/5
438613/438613 [==============================] - 6959s 16ms/step - loss: 0.1205 - accuracy: 0.9725
Epoch 5/5
438613/438613 [==============================] - 6980s 16ms/step - loss: 0.1352 - accuracy: 0.9725
34114.13438081741


In [54]:
model.save('trans_DDtoD_5.h5')

In [55]:
y_pred = model.predict(X_test)
y_pred = y_pred.argmax(axis=1)

In [56]:
rf_conf_mat = confusion_matrix(Y_test, y_pred)
print("Transfer learning confusion matrix: \n", rf_conf_mat)

Transfer learning confusion matrix: 
 [[  17049      18       0     251       2       0       0]
 [    610 1204577     270      17    1939       0       0]
 [    187       8 1846166      28   20599   42406       0]
 [    980      13    1471 1411042     231       0       0]
 [     18      97   24920       7 1213773       0       0]
 [    130       0    6859      40       3  625757       0]
 [    157   60945     261      60      98       0       0]]


In [57]:
CR = classification_report(Y_test,y_pred)
print(CR)

C:\Users\LabUser\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\LabUser\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.89      0.98      0.94     17320
           1       0.95      1.00      0.97   1207413
           2       0.98      0.97      0.97   1909394
           3       1.00      1.00      1.00   1413737
           4       0.98      0.98      0.98   1238815
           5       0.94      0.99      0.96    632789
           6       0.00      0.00      0.00     61521

    accuracy                           0.97   6480989
   macro avg       0.82      0.85      0.83   6480989
weighted avg       0.97      0.97      0.97   6480989



C:\Users\LabUser\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
